In [4]:
import numpy as np
import pandas as pd




In [5]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link,header=0)[3]
df3 = pd.read_html(link,header=0)[4]
df4 = pd.read_html(link,header=0)[5]

In [6]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\ajayj\AppData\Local\Temp\ipykernel_3728\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures / GFM Films,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[164]
270,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
271,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[229]
272,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[262]


In [8]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '5492165c61b1a21c06eb3a3b578a6339'

In [9]:
from tmdbv3api import Movie


tmdb_movie = Movie()

def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if result:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)  
    return np.NaN

In [ ]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df_2018 = df[['Title','Cast and crew','genres']]

In [ ]:
df_2018

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [ ]:
df_2018

In [ ]:

df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [ ]:
new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df18

In [ ]:
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
new_df18['movie_title'] = new_df18['movie_title'].str.lower()

In [ ]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']

In [ ]:
new_df18

## Extracting Features of 2019 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]
df5 = pd.read_html(link, header=0)[2]

In [ ]:
df = df5.append(df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True), ignore_index = True),ignore_index = True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df['genres'] = df['Title'].map(lambda x : get_genre(str(x)))

In [ ]:
df_2019 = df[['Title','Cast and crew','genres']]

In [ ]:
df_2019

In [ ]:
df.loc[: ,'Cast and crew']

In [ ]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
df_2019['Cast and crew'] = df_2019['Cast and crew'].fillna('')


In [ ]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))

In [ ]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))

In [ ]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [ ]:
df_2019 = df_2019.rename(columns = {'Title':'movie_title'})

In [ ]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan,'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan,'unknown')

In [ ]:
new_df19['movie_title'] = new_df19['movie_title'].str.lower()


In [ ]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']

In [ ]:
new_df19

In [ ]:
my_df = new_df18.append(new_df19,ignore_index = True)

In [ ]:
my_df

In [ ]:
old_df = pd.read_csv('movie.csv')

In [ ]:
old_df

In [ ]:
final_df = old_df.append(my_df,ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.isna().sum()

In [ ]:
final_df=final_df.dropna(how="any")

In [ ]:
final_df

In [ ]:
final_df.to_csv('final_data.csv',index=False)

## Extracting data of 2020 movies


In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]
df5 = pd.read_html(link, header=0)[2]

In [ ]:
df = df5.append(df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True),ignore_index=True)

In [ ]:
df

In [ ]:
df_2020 = df[['Title','Cast and crew']]

In [ ]:
df_2020

In [ ]:
df_2020['genres'] = df_2020['Title'].map(lambda x : get_genre(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020['director_name'] = df_2020['Title'].map(lambda x: get_director(str(x)))

In [ ]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [ ]:
new_df20

## Extracting data from 2021

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df5 = pd.read_html(link, header=0)[6]

In [ ]:
df = df1.append(df2.append(df3.append(df4.append(df5, ignore_index = True),ignore_index = True),ignore_index = True),ignore_index=True)

In [ ]:
df

In [ ]:
df_2021 = df[['Title','Cast and crew']]

In [ ]:
df_2021

In [ ]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2021

In [ ]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
df_2021

In [ ]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [ ]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df21

In [ ]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [ ]:
new_df21

In [ ]:
new_df21 = new_df21.dropna(how = "any")

In [ ]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

In [ ]:
new_df21

## Extracting data from 2022

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
df1 = pd.read_html(link , header=0)[3]
df2 = pd.read_html(link , header=0)[4]
df3 = pd.read_html(link , header=0)[5]
df4 = pd.read_html(link , header=0)[6]

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index = True),ignore_index=True),ignore_index=True)

In [ ]:
df

In [ ]:
df_2022 = df[['Title','Cast and crew']]

In [ ]:
df_2022

In [ ]:
df_

In [ ]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2022

In [ ]:
df_2022 = df_2022.rename(columns = {'Title':'movie_title'})

In [ ]:
df_2022['genres'] = df_2022['movie_title'].map(lambda x: get_genre(str(x)))

In [ ]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df22

In [ ]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [ ]:
new_df22 = new_df22.dropna(how="any")

In [ ]:
new_df22

## Exrtacting data of 2023

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"
df1 = pd.read_html(link , header=0)[3]
df2 = pd.read_html(link , header=0)[4]
df3 = pd.read_html(link , header=0)[5]
df4 = pd.read_html(link , header=0)[6]

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [ ]:
df

In [ ]:
df_2023 = df[['Title','Cast and crew']]

In [ ]:
df_2023

In [ ]:
df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
df_2023['actor_2_name'] = df_2023['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2023

In [ ]:
df_2023 = df_2023.rename(columns = {'Title':'movie_title'})

In [ ]:
df_2023

In [ ]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df23

In [ ]:
new_df23['comb'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']

In [ ]:
new_df23 = new_df23.dropna(how="any")

In [ ]:
new_df23

In [ ]:
new_df23['movie_title'] = new_df23['movie_title'].str.lower()

In [ ]:
new_df23

## Combining all the 20,21,22,23 csv

In [ ]:
final_df = final_df.append(new_df20.append(new_df21.append(new_df22.append(new_df23,ignore_index=True),ignore_index=True),ignore_index=True),ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv('main_data.csv')

In [ ]:
 movie_combination = final_df.iloc[movie_index]['comb']